In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

<h2>Define Hyper-parameters</h2>

In [13]:
num_iter = 10000 #training iterations
lr = 0.2

<h2>Define your input data and expected labels</h2>

In [14]:
# define input data and XOR gate outputs
x = np.array([[0,0], [0,1], [1, 0], [1,1]], np.float32)
y = np.array([[0], [1], [1] , [0]], np.float32)

<h2>Define placeholder and Variables</h2>

In [15]:
tf.reset_default_graph() # reset
# placeholders for feeding
X_in = tf.placeholder(tf.float32, shape=(4,2))
Y_hat = tf.placeholder(tf.float32, shape=(4,1))

# neural net vars
# hidden layer
w1 = tf.Variable(tf.random_normal(shape=[2,2]), tf.float32)
b1 = tf.Variable(tf.zeros([1,1]),tf.float32)

# output layer
wout = tf.Variable(tf.random_normal(shape=[2,1]), tf.float32)
bout = tf.Variable(tf.zeros([1,1]), tf.float32)

<h2>Define the neural network</h2>

In [16]:
h_lay_out = tf.sigmoid(tf.add(tf.matmul(X_in, w1), b1)) # single hidden layer with sigmoid activation
output = tf.add(tf.matmul(h_lay_out, wout), bout) # output layer, scalar, sigmoid for classification

<h2>Define cost function</h2>

In [17]:
# cross entropy as cost
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=output,labels=Y_hat))

#define loss
optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
train_op = optimizer.minimize(loss)

#compare the predicted labels with true labels (classification is round)
correct_pred = tf.equal(tf.round(tf.sigmoid(output)),Y_hat)

#compute the accuracy by taking average
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32),name='accuracy')

#Initialize the variables
init = tf.global_variables_initializer()

<h2>Execute training</h2>

In [19]:
with tf.Session() as sess:
    sess.run(init)

    for i in range(num_iter):
        sess.run(train_op, feed_dict={X_in: x, Y_hat: y})
        if i % 1000 == 0:
            # output
            #temp = sess.run(output,feed_dict={X_in: x, Y_hat: y})
            #print(temp)
            #print('Output sigmoid, which we use to classify')
            #print(sess.run(tf.nn.sigmoid(temp)))
            #print('True Labels')
            #print(y)
            #print('Loss')
            print(sess.run(loss, feed_dict={X_in: x, Y_hat: y}))
            print(sess.run(correct_pred, feed_dict={X_in: x, Y_hat: y}))
            acc = sess.run(accuracy,feed_dict={X_in: x, Y_hat: y})
            print("step "+str(i)+", Accuracy= {:.3f}".format(acc))
    # its possible to generate a decision boundary by evaluating a dense grid
    # this is easier than solving the nonlinear function
    # honestly must be easier way to do this
    
    print("Training finished!")
    

0.7146411
[[ True]
 [False]
 [False]
 [ True]]
step 0, Accuracy= 0.500
0.59417266
[[ True]
 [ True]
 [ True]
 [False]]
step 1000, Accuracy= 0.750
0.16638735
[[ True]
 [ True]
 [ True]
 [ True]]
step 2000, Accuracy= 1.000
0.05089415
[[ True]
 [ True]
 [ True]
 [ True]]
step 3000, Accuracy= 1.000
0.027662884
[[ True]
 [ True]
 [ True]
 [ True]]
step 4000, Accuracy= 1.000
0.01860083
[[ True]
 [ True]
 [ True]
 [ True]]
step 5000, Accuracy= 1.000
0.013891155
[[ True]
 [ True]
 [ True]
 [ True]]
step 6000, Accuracy= 1.000
0.011035433
[[ True]
 [ True]
 [ True]
 [ True]]
step 7000, Accuracy= 1.000
0.009129606
[[ True]
 [ True]
 [ True]
 [ True]]
step 8000, Accuracy= 1.000
0.0077718683
[[ True]
 [ True]
 [ True]
 [ True]]
step 9000, Accuracy= 1.000
Training finished!


<h2>Plot</h2>

In [21]:
x1 = np.linspace(0,1,1000)
x2 = np.linspace(0,1,1000)
X, Y = np.meshgrid(x1, x2)

(1000, 1000)